<a href="https://colab.research.google.com/github/ddesai-sjsu/Machine-Learning/blob/main/Homework_8_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Read Week 8 readings on the website:

 https://sites.google.com/sjsu.edu/appliedml/home?authuser=1 (Links to an external site.)
 **Done**



* Read the short description of your Veracity Factor. 

  **Sentiment Analysis** :: Sentiment Analysis (SA) is the branch of Natural Language Processing (NLP) in charge of the design and implementation of models, methods, and techniques to determine whether a text deals with objective or subjective information and, in the latter case, to determine if such information is expressed in a positive, neutral, or negative way as well as if it is expressed in a strong or weak way. Since a large part of the subjective content expressed by users on social networks is about opinions (on review sites, forums, message boards, chats, etc.), SA is also known as Opinion Mining (OM).


* Research your factor (google "factor + machine learning"), find an academic paper that describes how to implement or design your factor

  https://www.sciencedirect.com/science/article/pii/S095070512100397X

* identify at least 3 micro-factors from that paper

  **Terms presence and frequency:** 
  is the simplest way of representing features and it is commonly used for Information Retrieval as well as for sentiment analysis.In term frequency, a term is given an integer value that represents its count in the document. TF–IDF weighting scheme can be applied to measure the importance of the term in the document.

  **Parts-of-Speech (PoS) tags:**
  are the labels or annotations that identify the word’s function in a given language. In general, words can be categorized into several parts of speech categories

  https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

  **Opinion words and phrases:**
  opinion words are words that are commonly used to express positive or negative sentiments 
* implement at least 2 of them in python:

  Done below
* scrape politifact as a team project:

  Done below




## **Web scraping from Polifact:**

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [ ]:
#Create lists to store the scraped data
authors = []
dates = []
statements = []
sources = []
targets = []

In [ ]:
#Create a function to scrape the site
def scrape_website(page_number):
  page_num = str(page_number) #Convert the page number to a string
  URL = 'https://www.politifact.com/factchecks/list/?page='+page_num #append the page number to complete the URL
  webpage = requests.get(URL)  #Make a request to the website
  #time.sleep(3)
  soup = BeautifulSoup(webpage.text, "html.parser") #Parse the text from the website
  #Get the tags and it's class
  statement_footer =  soup.find_all('footer',attrs={'class':'m-statement__footer'})  #Get the tag and it's class
  statement_quote = soup.find_all('div', attrs={'class':'m-statement__quote'}) #Get the tag and it's class
  statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta'})#Get the tag and it's class
  target = soup.find_all('div', attrs={'class':'m-statement__meter'}) #Get the tag and it's class
  #loop through the footer class m-statement__footer to get the date and author
  for i in statement_footer:
    link1 = i.text.strip()
    name_and_date = link1.split()
    first_name = name_and_date[1]
    last_name = name_and_date[2]
    full_name = first_name+' '+last_name
    month = name_and_date[4]
    day = name_and_date[5]
    year = name_and_date[6]
    date = month+' '+day+' '+year
    dates.append(date)
    authors.append(full_name)
  #Loop through the div m-statement__quote to get the link
  for i in statement_quote:
    link2 = i.find_all('a')
    statements.append(link2[0].text.strip())
 #Loop through the div m-statement__meta to get the source
  for i in statement_meta:
    link3 = i.find_all('a') #Source
    source_text = link3[0].text.strip()
    sources.append(source_text)
  #Loop through the target or the div m-statement__meter to get the facts about the statement (True or False)
  for i in target:
    fact = i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
    targets.append(fact)

In [ ]:
#Loop through 'n-1' webpages to scrape the data
n=50
for i in range(1, n):
  scrape_website(i)

In [ ]:
#Create a new dataFrame 
data = pd.DataFrame(columns = ['author',  'statement', 'source', 'date', 'target']) 
data['author'] = authors
data['statement'] = statements
data['source'] = sources
data['date'] = dates
data['target'] = targets
#Show the data set

data.to_csv('politifact.csv', index=False)
data

,author,statement,source,date,target
0,Monique Curet,President Joe Biden threatened to “swoop down ...,Facebook posts,"November 9, 2021",false
1,Samantha Putterman,“More fraud in New Jersey’s election uncovered...,Bloggers,"November 9, 2021",barely-true
2,Samantha Putterman,Merriam-Webster recently changed its definitio...,Facebook posts,"November 9, 2021",barely-true
3,Emily Zirkelbach,“FUN FACT: West Virginia is home to zero billi...,David Gura,"November 9, 2021",true
4,Ciara O'Rourke,The deaths at the Astroworld festival were par...,Facebook posts,"November 9, 2021",pants-fire
...,...,...,...,...,...
1465,Ciara O'Rourke,“President-elect Biden steps down as president...,Viral image,"January 5, 2021",pants-fire
1466,Ciara O'Rourke,“1776 flag flying over White House!” in Januar...,Bloggers,"January 5, 2021",false
1467,Haley BeMiller,On whether he’ll object to the Electoral Colle...,Ron Johnson,"January 5, 2021",full-flop
1468,Samantha Putterman,New York “is trying to pass a bill that would ...,Facebook posts,"January 5, 2021",false


## **Implementation of Microfactors:**


### **1. Term presence & frequency**

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df=data
df['statement']=df['statement'].astype(str)
df['Lower']=df['statement'].str.lower()
df['Lower']=df['Lower'].str.replace('\d+', '')
df['Lower']=df['Lower'].str.replace(r"http\S+", '')
df['Lower']=df['Lower'].str.replace('[^A-Za-z0-9 ]+', '')
df

,author,statement,source,date,target,Lower,statement_without_stopwords,lemm,stemmed
0,Kiannah Sepeda-Miller,"The migrant caravan from Tapachula, Mexico “is...",Mary Miller,"October 31, 2021",pants-fire,the migrant caravan from tapachula mexico is t...,migrant caravan tapachula mexico population mi...,migrant caravan tapachula mexico population mi...,migrant caravan tapachula mexico popul minneapoli
1,Tom Kertscher,Blood “starts to clot in a minute or two” afte...,Instagram posts,"October 29, 2021",false,blood starts to clot in a minute or two after ...,blood starts clot minute two pfizer covid vacc...,blood start clot minute two pfizer covid vacci...,blood start clot minut two pfizer covid vaccin...
2,Monique Curet,“Japan drops vax rollout”,Hal Turner Radio Show,"October 29, 2021",pants-fire,japan drops vax rollout,japan drops vax rollout,japan drop vax rollout,japan drop vax rollout
3,Samantha Putterman,Virginia voters are being turned away from pol...,Facebook posts,"October 29, 2021",barely-true,virginia voters are being turned away from pol...,virginia voters turned away polling locations ...,virginia voter turned away polling location to...,virginia voter turn away poll locat told alrea...
4,Ciara O'Rourke,In-N-Out cups say “F--- Joe Biden.”,Viral image,"October 29, 2021",false,innout cups say f joe biden,innout cups say f joe biden,innout cup say f joe biden,innout cup say f joe biden
...,...,...,...,...,...,...,...,...,...
1465,Tom Kertscher,"Says Kelly Loeffler supports ""raising taxes on...",Raphael Warnock,"December 16, 2020",false,says kelly loeffler supports raising taxes on ...,says kelly loeffler supports raising taxes geo...,say kelly loeffler support raising tax georgia...,say kelli loeffler support rais tax georgia mi...
1466,Paul Egan,“68% error rate in Michigan Voting Machines. S...,Donald Trump,"December 15, 2020",false,error rate in michigan voting machines should...,error rate michigan voting machines law tiny p...,error rate michigan voting machine law tiny pe...,error rate michigan vote machin law tini perce...
1467,Victoria Knight,The percentage of Americans with natural immun...,Donald Trump,"December 15, 2020",false,the percentage of americans with natural immun...,percentage americans natural immunity getting ...,percentage american natural immunity getting c...,percentag american natur immun get covid power...
1468,Bill McCarthy,An optional microchip for COVID-19 vaccines “w...,Facebook posts,"December 15, 2020",false,an optional microchip for covid vaccines would...,optional microchip covid vaccines would give o...,optional microchip covid vaccine would give of...,option microchip covid vaccin would give offic...


In [ ]:
stop_words = set(stopwords.words('english')) 
df['statement_without_stopwords'] = df['Lower'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df

,author,statement,source,date,target,Lower,statement_without_stopwords,lemm,stemmed
0,Kiannah Sepeda-Miller,"The migrant caravan from Tapachula, Mexico “is...",Mary Miller,"October 31, 2021",pants-fire,the migrant caravan from tapachula mexico is t...,migrant caravan tapachula mexico population mi...,migrant caravan tapachula mexico population mi...,migrant caravan tapachula mexico popul minneapoli
1,Tom Kertscher,Blood “starts to clot in a minute or two” afte...,Instagram posts,"October 29, 2021",false,blood starts to clot in a minute or two after ...,blood starts clot minute two pfizer covid vacc...,blood start clot minute two pfizer covid vacci...,blood start clot minut two pfizer covid vaccin...
2,Monique Curet,“Japan drops vax rollout”,Hal Turner Radio Show,"October 29, 2021",pants-fire,japan drops vax rollout,japan drops vax rollout,japan drop vax rollout,japan drop vax rollout
3,Samantha Putterman,Virginia voters are being turned away from pol...,Facebook posts,"October 29, 2021",barely-true,virginia voters are being turned away from pol...,virginia voters turned away polling locations ...,virginia voter turned away polling location to...,virginia voter turn away poll locat told alrea...
4,Ciara O'Rourke,In-N-Out cups say “F--- Joe Biden.”,Viral image,"October 29, 2021",false,innout cups say f joe biden,innout cups say f joe biden,innout cup say f joe biden,innout cup say f joe biden
...,...,...,...,...,...,...,...,...,...
1465,Tom Kertscher,"Says Kelly Loeffler supports ""raising taxes on...",Raphael Warnock,"December 16, 2020",false,says kelly loeffler supports raising taxes on ...,says kelly loeffler supports raising taxes geo...,say kelly loeffler support raising tax georgia...,say kelli loeffler support rais tax georgia mi...
1466,Paul Egan,“68% error rate in Michigan Voting Machines. S...,Donald Trump,"December 15, 2020",false,error rate in michigan voting machines should...,error rate michigan voting machines law tiny p...,error rate michigan voting machine law tiny pe...,error rate michigan vote machin law tini perce...
1467,Victoria Knight,The percentage of Americans with natural immun...,Donald Trump,"December 15, 2020",false,the percentage of americans with natural immun...,percentage americans natural immunity getting ...,percentage american natural immunity getting c...,percentag american natur immun get covid power...
1468,Bill McCarthy,An optional microchip for COVID-19 vaccines “w...,Facebook posts,"December 15, 2020",false,an optional microchip for covid vaccines would...,optional microchip covid vaccines would give o...,optional microchip covid vaccine would give of...,option microchip covid vaccin would give offic...


In [ ]:

# Use English stemmer.
stemmer = SnowballStemmer("english")
lem=WordNetLemmatizer()
df['lemm'] = df['statement_without_stopwords'].apply(lambda x: ' '.join([lem.lemmatize(y) for y in x.split()]))
df['stemmed'] = df['lemm'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split()]))
df

,author,statement,source,date,target,Lower,statement_without_stopwords,lemm,stemmed
0,Kiannah Sepeda-Miller,"The migrant caravan from Tapachula, Mexico “is...",Mary Miller,"October 31, 2021",pants-fire,the migrant caravan from tapachula mexico is t...,migrant caravan tapachula mexico population mi...,migrant caravan tapachula mexico population mi...,migrant caravan tapachula mexico popul minneapoli
1,Tom Kertscher,Blood “starts to clot in a minute or two” afte...,Instagram posts,"October 29, 2021",false,blood starts to clot in a minute or two after ...,blood starts clot minute two pfizer covid vacc...,blood start clot minute two pfizer covid vacci...,blood start clot minut two pfizer covid vaccin...
2,Monique Curet,“Japan drops vax rollout”,Hal Turner Radio Show,"October 29, 2021",pants-fire,japan drops vax rollout,japan drops vax rollout,japan drop vax rollout,japan drop vax rollout
3,Samantha Putterman,Virginia voters are being turned away from pol...,Facebook posts,"October 29, 2021",barely-true,virginia voters are being turned away from pol...,virginia voters turned away polling locations ...,virginia voter turned away polling location to...,virginia voter turn away poll locat told alrea...
4,Ciara O'Rourke,In-N-Out cups say “F--- Joe Biden.”,Viral image,"October 29, 2021",false,innout cups say f joe biden,innout cups say f joe biden,innout cup say f joe biden,innout cup say f joe biden
...,...,...,...,...,...,...,...,...,...
1465,Tom Kertscher,"Says Kelly Loeffler supports ""raising taxes on...",Raphael Warnock,"December 16, 2020",false,says kelly loeffler supports raising taxes on ...,says kelly loeffler supports raising taxes geo...,say kelly loeffler support raising tax georgia...,say kelli loeffler support rais tax georgia mi...
1466,Paul Egan,“68% error rate in Michigan Voting Machines. S...,Donald Trump,"December 15, 2020",false,error rate in michigan voting machines should...,error rate michigan voting machines law tiny p...,error rate michigan voting machine law tiny pe...,error rate michigan vote machin law tini perce...
1467,Victoria Knight,The percentage of Americans with natural immun...,Donald Trump,"December 15, 2020",false,the percentage of americans with natural immun...,percentage americans natural immunity getting ...,percentage american natural immunity getting c...,percentag american natur immun get covid power...
1468,Bill McCarthy,An optional microchip for COVID-19 vaccines “w...,Facebook posts,"December 15, 2020",false,an optional microchip for covid vaccines would...,optional microchip covid vaccines would give o...,optional microchip covid vaccine would give of...,option microchip covid vaccin would give offic...


In [ ]:
df_final=df[['stemmed','target']]
df_final=df_final.rename(columns={'stemmed': "PreProcessed Statement"})
df_final

,PreProcessed Statement,target
0,migrant caravan tapachula mexico popul minneapoli,pants-fire
1,blood start clot minut two pfizer covid vaccin...,false
2,japan drop vax rollout,pants-fire
3,virginia voter turn away poll locat told alrea...,barely-true
4,innout cup say f joe biden,false
...,...,...
1465,say kelli loeffler support rais tax georgia mi...,false
1466,error rate michigan vote machin law tini perce...,false
1467,percentag american natur immun get covid power...,false
1468,option microchip covid vaccin would give offic...,false


In [ ]:

tfidf_vect = TfidfVectorizer(max_features=300)
tfidf = tfidf_vect.fit_transform(df_final['PreProcessed Statement'].values)
tfidf

<1470x300 sparse matrix of type '<class 'numpy.float64'>'
	with 6923 stored elements in Compressed Sparse Row format>

### **2. Parts of Speech(PoS) tags**

In [ ]:
from collections import Counter
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
def get_POS_tags(statement):
  tokens = nltk.word_tokenize(statement)
  text = nltk.Text(tokens)
  tags = nltk.pos_tag(text)
  counts = Counter(tag for word,tag in tags)
  noun_counts=counts['NN']+counts['NNS']+counts['NNP']+counts['NNPS']
  verb_counts=counts['VB']+counts['VBD']+counts['VBG']+counts['VBN']+counts['VBP']+counts['VBG']
  return noun_counts,verb_counts

In [ ]:
df_final['noun_counts']=df_final['PreProcessed Statement'].apply(lambda x:get_POS_tags(x)[0])

In [ ]:
df_final['verb_counts']=df_final['PreProcessed Statement'].apply(lambda x:get_POS_tags(x)[1])

In [ ]:
df_final

,PreProcessed Statement,target,noun_counts,verb_counts
0,migrant caravan tapachula mexico popul minneapoli,pants-fire,5,0
1,blood start clot minut two pfizer covid vaccin...,false,7,1
2,japan drop vax rollout,pants-fire,4,0
3,virginia voter turn away poll locat told alrea...,barely-true,7,2
4,innout cup say f joe biden,false,3,1
...,...,...,...,...
1465,say kelli loeffler support rais tax georgia mi...,false,7,1
1466,error rate michigan vote machin law tini perce...,false,6,1
1467,percentag american natur immun get covid power...,false,5,0
1468,option microchip covid vaccin would give offic...,false,9,1
